In [1]:
import pyspark
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.shell import sqlContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, substring
from pyspark.sql import Window
from pyspark.sql.functions import dense_rank
from pyspark.sql.functions import row_number
import pandas as pd

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.0.2.6.5.1100-53
      /_/

Using Python version 3.6.5 (default, Apr  6 2018 13:39:56)
SparkSession available as 'spark'.


In [6]:
df=spark.read.json('/ge/mkt/appannie/*/downloads_and_revenue/*')
a=df.select(df.product_id,df.product_name,df.feed, df.estimate,df.country,df.end_date,df.publisher_id,df.publisher_name,df.parent_company_id,df.parent_company_name)

In [7]:
a.show(5)

+--------------+--------------------+----+--------+-------+----------+--------------+----------------+-----------------+-------------------+
|    product_id|        product_name|feed|estimate|country|  end_date|  publisher_id|  publisher_name|parent_company_id|parent_company_name|
+--------------+--------------------+----+--------+-------+----------+--------------+----------------+-----------------+-------------------+
|20600010081528|         Brawl Stars|free|    7723|     JP|2019-01-12|20200000120379|       Supercell| 1000200000016673|          Supercell|
|20600010928153| 覇穹 封神演義 ～センカイクロニクル～|free|    5152|     JP|2019-01-12|20200002424223|    GMO GP, Inc.| 1000200000003514|                GMO|
|20600001938601|     LINE：ディズニー ツムツム|free|    4945|     JP|2019-01-12|20200000201193|LINE Corporation| 1000200000037673|               LINE|
|20600011010555|               非人類学園|free|    4781|     JP|2019-01-12|20200002497273|     X.D. Global| 1000200000021757|       X.D. Network|
|206000052396

In [10]:
x=a.select(a.product_id,a.product_name,a.country,a.publisher_id,a.publisher_name, a.end_date).distinct()
gg= Window.partitionBy("country","product_ID").orderBy(x.end_date.desc())
y=x.withColumn("rank",row_number().over(gg))
xy=y.filter(y.rank=='1')
xy.show(10)

+----------+--------------------+-------+------------+--------------------+----------+----+
|product_id|        product_name|country|publisher_id|      publisher_name|  end_date|rank|
+----------+--------------------+-------+------------+--------------------+----------+----+
| 321366355|      Rat On The Run|     CN|   310068844|         Donut Games|2020-11-03|   1|
| 358327712|Airport Terminal ...|     CN|   309411049|Meridian Digital ...|2020-02-24|   1|
| 404410628|         Winterbells|     CN|   400047223|         Ferry Halim|2020-08-07|   1|
| 494302506|       Scotland Yard|     CN|  1505283467|Ravensburger Verl...|2020-11-03|   1|
| 500108743|       仙劍奇俠傳5 - 劍傲丹楓|     CN|   443885994|SOFTSTAR ENTERTAI...|2020-11-07|   1|
| 512142109|    Max Payne Mobile|     CN|   330049731|      Rockstar Games|2020-11-07|   1|
| 547167853|        Groove Catch|     CN|   438953814|  WG Publishing Inc.|2018-09-08|   1|
| 547174523|         Blast-A-Way|     CN|   297694604|       Illusion Labs|2020-

In [11]:
yz=xy.select(xy.product_id, xy.product_name, xy.country, xy.publisher_id,xy.publisher_name, xy.end_date)

In [16]:
aa=a.join(yz,on=['product_id','country','end_date','publisher_id','product_name','publisher_name'],how='left')

In [17]:
aa.show(5)

+----------+-------+----------+------------+------------+--------------------+----+--------+-----------------+-------------------+
|product_id|country|  end_date|publisher_id|product_name|      publisher_name|feed|estimate|parent_company_id|parent_company_name|
+----------+-------+----------+------------+------------+--------------------+----+--------+-----------------+-------------------+
| 281736535|     ID|2019-09-20|   281736538|      Enigmo|Pangea Software, ...|paid|       1| 1000200000021607|    Pangea Software|
| 281736535|     ID|2020-09-08|   281736538|      Enigmo|Pangea Software, ...|paid|       1| 1000200000021607|    Pangea Software|
| 281736535|     JP|2019-05-25|   281736538|      Enigmo|Pangea Software, ...|paid|       2| 1000200000021607|    Pangea Software|
| 281736535|     KR|2019-05-20|   281736538|      Enigmo|Pangea Software, ...|paid|       1| 1000200000021607|    Pangea Software|
| 281736535|     MY|2019-12-23|   281736538|      Enigmo|Pangea Software, ...|paid|

In [18]:
bb=aa.withColumn('month', aa.end_date.substr(1, 7))

In [19]:
bb.show(5)

+----------+-------+----------+------------+------------+--------------------+----+--------+-----------------+-------------------+-------+
|product_id|country|  end_date|publisher_id|product_name|      publisher_name|feed|estimate|parent_company_id|parent_company_name|  month|
+----------+-------+----------+------------+------------+--------------------+----+--------+-----------------+-------------------+-------+
| 281736535|     ID|2019-09-20|   281736538|      Enigmo|Pangea Software, ...|paid|       1| 1000200000021607|    Pangea Software|2019-09|
| 281736535|     ID|2020-09-08|   281736538|      Enigmo|Pangea Software, ...|paid|       1| 1000200000021607|    Pangea Software|2020-09|
| 281736535|     JP|2019-05-25|   281736538|      Enigmo|Pangea Software, ...|paid|       2| 1000200000021607|    Pangea Software|2019-05|
| 281736535|     KR|2019-05-20|   281736538|      Enigmo|Pangea Software, ...|paid|       1| 1000200000021607|    Pangea Software|2019-05|
| 281736535|     MY|2019-12

In [33]:
b=bb.filter(bb.feed=='free').withColumnRenamed("estimate","install").drop('feed')
install=b.groupBy('month','country','product_id','publisher_id','product_name','publisher_name').agg(F.sum('install').alias('install'))

In [63]:
c=bb.filter(bb.feed=='grossing').withColumnRenamed("estimate","revenue").drop('feed')
revenue=c.groupBy('month','country','product_id','publisher_id','product_name','publisher_name').agg(F.sum('revenue').alias('revenue'))

In [64]:
downloads_and_revenue=install.join(revenue, on= ['product_name','month','country','product_id','publisher_id','publisher_name','product_name'], how='left')

In [65]:
downloads_and_revenue.show(5)

+--------------------+-------+-------+----------+------------+-------------------+--------------------+-------+-------+
|        product_name|  month|country|product_id|publisher_id|     publisher_name|        product_name|install|revenue|
+--------------------+-------+-------+----------+------------+-------------------+--------------------+-------+-------+
|#1 Siren Head Cal...|2020-08|     PH|1524300865|  1522196735|    Diane Pedneault|#1 Siren Head Cal...|     39|   null|
|              #DRIVE|2019-05|     MY|1366180154|   498490654|   Dariusz Pietrala|              #DRIVE|   1054|   null|
|           #SelfCare|2020-09|     PH|1378384555|  1176266442|Tru Luv Media, Inc.|           #SelfCare|    402|     10|
|           #SelfCare|2020-09|     TH|1378384555|  1176266442|Tru Luv Media, Inc.|           #SelfCare|    244|   null|
|           .Connect.|2018-12|     ID|1416467796|  1146168572|       Kishor Berde|           .Connect.|    548|   null|
+--------------------+-------+-------+--

In [66]:
# TABLE_1

In [67]:
list_of_market = ['ID','MY','PH','SG','TH','TW','VN']
market = list_of_market[0]

In [68]:
monthly_ios = spark.read.json('/ge/mkt/appannie/monthly/*/usage_data/{}/ios.json'.format(market)).withColumn('platform', F.lit('ios'))
monthly_android = spark.read.json('/ge/mkt/appannie/monthly/*/usage_data/{}/android.json'.format(market)).withColumn('platform', F.lit('android'))
monthly = monthly_ios.union(monthly_android).withColumn('country', F.lit(market))

In [ ]:
for market in list_of_market[1:]:
    monthly_ios = spark.read.json('/ge/mkt/appannie/monthly/*/usage_data/{}/ios.json'.format(market)).withColumn('platform', F.lit('ios'))
    monthly_android = spark.read.json('/ge/mkt/appannie/monthly/*/usage_data/{}/android.json'.format(market)).withColumn('platform', F.lit('android'))
    monthly_new = monthly_ios.union(monthly_android).withColumn('country', F.lit(market))
    monthly = monthly.union(monthly_new)

In [ ]:
active_users=monthly.select(monthly.product_name,monthly.end_date,monthly.country,monthly.publisher_name,monthly.active_users,monthly.product_id,monthly.publisher_id,).withColumn("month", substring(monthly.end_date, 1, 7)).drop('end_date')

In [ ]:
database_new=downloads_and_revenue.join(active_users, on= ['month','country','product_id', 'publisher_id','product_name','publisher_name'], how='inner')

In [ ]:
h=spark.read.option("header",True).csv("map_appid_new.csv")
h.select(h.appannie_id,h.cat_1,h.cat_2,h.cat_3)
cat=h.selectExpr('appannie_id as product_id', 'cat_1 as category','cat_2 as category_VNG')

In [ ]:
table_all=database_new.join(cat, on= ['product_id'], how='inner')

In [ ]:
win1  = Window.partitionBy("country","month").orderBy(table_all.install.desc())
win2 = Window.partitionBy("country","month").orderBy(table_all.revenue.desc())
win3 = Window.partitionBy("country","month").orderBy(table_all.active_users.desc())
table_1=table_all.withColumn("rank_install", F.rank().over(win1)).withColumn("rank_revenue", F.rank().over(win2)).withColumn("rank_active_users", F.rank().over(win3))

In [ ]:
top_500_game=table_1.filter((table_1.rank_install<501)|(table_1.rank_revenue<501)|(table_1.rank_active_users<501))
top_500_game.show(20)

+--------------+-------+-------+--------------+--------------------+--------------------+-------+-------+------------+----------+---------------+------------+------------+-----------------+
|    product_id|  month|country|  publisher_id|        product_name|      publisher_name|install|revenue|active_users|  category|   category_VNG|rank_install|rank_revenue|rank_active_users|
+--------------+-------+-------+--------------+--------------------+--------------------+-------+-------+------------+----------+---------------+------------+------------+-----------------+
|20600006642345|2018-08|     VN|20200001720791|Garena Liên Quân ...|Garena Mobile Pri...|1204776| 133045|     7709146|    Esport|           MOBA|           1|           1|                1|
|20600008012481|2018-08|     VN|20200002303403|Mini World: Block...|        MiniPlay Inc| 497659|   8042|     2015636|    Action|      Adventure|          13|          32|                2|
|20600001704583|2018-08|     VN|20200002602721|   

In [ ]:
# TABLE_2

In [ ]:
k=table_all.select(table_all.publisher_id,table_all.publisher_name,table_all.country,table_all.install,table_all.revenue,table_all.active_users,table_all.month)
l=k.groupBy('month','country','publisher_id','publisher_name').sum('install','revenue','active_users')

In [ ]:
m = l.withColumnRenamed('sum(install)','install').withColumnRenamed('sum(revenue)','revenue').withColumnRenamed('sum(active_users)','active_users')

In [ ]:
win4  = Window.partitionBy("country","month").orderBy(m.install.desc())
win5 = Window.partitionBy("country","month").orderBy(m.revenue.desc())
win6 = Window.partitionBy("country","month").orderBy(m.active_users.desc())
table_2=m.withColumn("rank_install", F.rank().over(win4)).withColumn("rank_revenue", F.rank().over(win5)).withColumn("rank_active_users", F.rank().over(win6))

In [ ]:
top_50_pub=table_2.filter((table_2.rank_install<51)|(table_2.rank_revenue<51)|(table_2.rank_active_users<51))
top_50_pub.show(20)

In [ ]:
# TABLE_3

In [ ]:
table_VN=table_all.filter(df.country=='VN')
table_VN.show()

In [ ]:
#TABLE_4

In [ ]:
table_PH=table_all.filter(df.country=='PH')
table_PH.show()

In [ ]:
#TABLE_5

In [ ]:
table_TW=table_all.filter(df.country=='TW')
table_TW.show()

In [ ]:
#TABLE_6

In [ ]:
table_ID=table_all.filter(df.country=='ID')
table_ID.show()

In [ ]:
#TABLE_7

In [ ]:
table_SG=table_all.filter(df.country=='SG')
table_SG.show()

In [ ]:
#TABLE_8

In [ ]:
table_TH=table_all.filter(df.country=='TH')
table_TH.show()

In [ ]:
#TABLE_9

In [ ]:
table_MY=table_all.filter(df.country=='MY')
table_MY.show()